# Machine Learning Pipeline with KubeDirector - Lab 3
## Build, train and test your model in a remote tenant-shared training cluster 

### **Lab Workflow:**

In this lab: 

1. You will first create your tenant user's working directory on the shared persistent container storage needed in the ML pipeline to store the dataset, trained models and scoring scripts.

2. You will then build, train and test the model on a dataset using a remote tenant-shared training cluster. 

#### About the Dataset
The dataset is based on the 2019 New York City yellow cab trip data (approximately 375,000 trip records from January-June 2019). The dataset has many different properties (aka _"X - features"_) like the pickup time and location, the dropoff time and location, the trip distance, the number of passengers, and several other variables. The goal is to predict the taxi ride duration in NY City (the target aka _"Y - label"_) based on these features. For this workshop, you will be using a premade dataset that requires little preprocessing. 

#### About the Machine Learning Model algorithm
Gradient boosting supervised machine learning algorithm with Python is used here to build the model that is capable of predicting the duration of taxi trips in New York city. Python libraries such as Numpy, Pandas, Scikit-learn, XGBoost are used to build the model. 

The machine learning workflow depicted in the diagram below follows the typical supervised machine learning workflow:
- After loading the dataset, the ML algorithm separates data into features (the taxi ride properties) and label (the taxi ride duration). 
- Then the dataset is divided into two parts, one for training the model and one for testing the model. The typical split is 80/20.
- The ML algorithm is then defined and the model is built with the training data set to learn from. 
- Once the model is trained, the ML algorithm tests the model by making predictions on the test data set with the test features. 
- Next, the model accuracy is evaluated by comparing the test predictions to the test labels. Error metrics such as RMSE are used to evaluate the predictions accuracy.
- The trained model is finally saved to a file in the tenant user's working directory in the shared persistent container storage repository. In the lab part 4 of the workshop, the trained model will be loaded to the production deployment engine from the saved file to serve predictions on new data (Lab part 5 of the workshop).

![ML-Workflow](ML-Workflow.jpg)



### **Setup your working directory:**

The Python code here is used to create the working directory for your tenant userID. The working directory is used to store the key data components needed in the ML pipeline such as the input dataset, trained ML model(s) and scoring script(s). 

In [1]:
#userID = "student{{ STDID }}"

userID = "student74"

import os
import shutil
import time

# Shared dataset across all tenant users, use case directory name and scoring file name
datasetFile = "demodata.csv"
locationTable = "lookup-ipyheader.csv"
usecaseDirectory = "NYCTaxi"
scoringFile="XGB_Scoring.py"
scoringFileV2="XGB_Scoringv2.py"

# Project repo path function - file system mount available to all app containers
def ProjectRepo(path):
    ProjectRepo = "/bd-fs-mnt/TenantShare/repo"
    return str(ProjectRepo + '/' + path)


# Delete existing working directory for userID (that may exists from previous execution of the workshop)
studentRepoModel = "models" + '/' + usecaseDirectory + '/' + userID
studentRepoCode = "code" + '/' + usecaseDirectory + '/' + userID
pathModel = ProjectRepo(studentRepoModel)
pathCode = ProjectRepo(studentRepoCode)

if (os.path.exists(ProjectRepo(studentRepoModel))):
    print ("Repository " + pathModel + " exists for user " + userID + ". Deleting the repo now...")
    shutil.rmtree(pathModel, ignore_errors=True)
        
if (os.path.exists(ProjectRepo(studentRepoCode))):
    print ("Repository " + pathCode + " exists for user " + userID + ". Deleting the repo now...")
    shutil.rmtree(pathCode, ignore_errors=True)
    
# Making sure the input Dataset is loaded and accessible in the shared persistent container storage for your tenant
# Check the dataset file exists in /db-fs-mnt/TenantShare/repo/data/NYCTaxi folder:
#print (os.listdir(ProjectRepo('data/NYCTaxi')))
datasetFilePath = "data" + '/' + usecaseDirectory + '/' + datasetFile
locationFilePath = "data" + '/' + usecaseDirectory + '/' + locationTable
pathData = ProjectRepo("data" + '/' + usecaseDirectory)

if (not os.path.exists(ProjectRepo(datasetFilePath))):
    print ("Error! Dataset file " + ProjectRepo(datasetFilePath) + " does not exist. Please contact the workshop administrator at hpedev.hackshack@hpe.com before continuing.")
if (not os.path.exists(ProjectRepo(locationFilePath))):
    print ("Error! location table file " + ProjectRepo(locationFilePath) + " does not exist. Please contact the workshop administrator at hpedev.hackshack@hpe.com before continuing.")


Repository /bd-fs-mnt/TenantShare/repo/models/NYCTaxi/student74 exists for user student74. Deleting the repo now...
Repository /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74 exists for user student74. Deleting the repo now...


**Creating your working directory structure to store the trained models and scoring code scripts**

In [2]:
# Define the directory structure for the UserID to store trained model and the scoring script
# /bd-fs-mnt/TenantShare/repo/models/NYCTaxi/userID; /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/userID

if (not os.path.exists(ProjectRepo(studentRepoModel))):
    print ("Model repository " + pathModel + " does not exist for user. Creating the directory now...")
    try:
        os.makedirs(pathModel)
    except OSError:
        print ("Creation of the model directory %s failed" % pathModel)
    else:
        print ("Successfully created the model directory %s" % pathModel)

if (not os.path.exists(ProjectRepo(studentRepoCode))):
    print ("Code repository for user " + pathCode + " does not exist for user. Creating the directory now...")
    try:
        os.makedirs(pathCode)
    except OSError:
        print ("Creation of the code directory %s failed" % pathCode)
    else:
        print ("Successfully created the code directory %s" % pathCode)
        
# Copying scoring script files to your code repository in your working directory
# Make sure the scoring script files are available in the /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/userID folder with appropriate permissions (read-execute)
##print ("local directory on your local Jupyter Notebook:")
##print (os.listdir(os.curdir))
##print ("target directory on the shared File System mount for your tenant: ")
##print (os.listdir(pathCode))
srcFile = scoringFile
destFile = pathCode + '/' + scoringFile
srcFileV2 = scoringFileV2
destFileV2 = pathCode + '/' + scoringFileV2

if (not os.path.exists(scoringFile)):
    print (scoringFile + " does not exist in your local Jupyter Notebook! Please make sure to copy the file " + scoringFile + " to your local Jupyter Notebook, then re-run that code cell to continue the lab.")
else:    
    if (os.path.exists(pathCode + '/' + scoringFile)):
        print (pathCode + '/' + scoringFile + " file exists. Setting permissions")
        os.chmod (destFile, 0o777)
    else:
        print ("File" + pathCode + '/' + scoringFile + " does not exist.")
        print ("Copying it now on " + pathCode + " and setting file permissions")
        try:
            shutil.copy(srcFile, destFile)
            os.chmod (destFile, 0o777)
        except IOError as e:
            print ("Unable to copy scoring file. %s" % e )
        except:
            print ("Unexpected error:", sys.exec_info())

if (not os.path.exists(scoringFileV2)):
    print (scoringFileV2 + " does not exist in your local Jupyter Notebook! Please make sure to copy the file " + scoringFileV2 + " to your local Jupyter Notebook, then re-run that code cell to continue the lab.")
else:    
    if (os.path.exists(pathCode + '/' + scoringFileV2)):
        print (pathCode + '/' + scoringFileV2 + " file exists. Setting permissions")
        os.chmod (destFileV2, 0o777)
    else:
        print ("File" + pathCode + '/' + scoringFileV2 + " does not exist.")
        print ("Copying it now on " + pathCode + " and setting file permissions")
        try:
            shutil.copy(srcFileV2, destFileV2)
            os.chmod (destFileV2, 0o777)
        except IOError as e:
            print ("Unable to copy scoring file. %s" % e )
        except:
            print ("Unexpected error:", sys.exec_info())
            
print ("target directory  " + pathCode + " :")
print (os.listdir(pathCode))


Model repository /bd-fs-mnt/TenantShare/repo/models/NYCTaxi/student74 does not exist for user. Creating the directory now...
Successfully created the model directory /bd-fs-mnt/TenantShare/repo/models/NYCTaxi/student74
Code repository for user /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74 does not exist for user. Creating the directory now...
Successfully created the code directory /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74
File/bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74/XGB_Scoring.py does not exist.
Copying it now on /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74 and setting file permissions
File/bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74/XGB_Scoringv2.py does not exist.
Copying it now on /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74 and setting file permissions
target directory  /bd-fs-mnt/TenantShare/repo/code/NYCTaxi/student74 :
['XGB_Scoringv2.py', 'XGB_Scoring.py']


### **Test Connection to the tenant-shared Training Cluster:**

Next, you'll test that the communication with the shared training cluster is indeed functioning properly.

> **Note:** _The KubeDirectorApp Jupyter Notebook used in our solution includes **custom magic** functions to handle remotely submitting training code and retrieving results and logs. The Notebook uses these magic functions to make REST API calls to the API server that runs as part of the shared training environment. These calls submit training jobs and get results from within the Notebook session._

The Jupyter Notebook kdapp includes the following **line magic** functions: 
*	%attachments: Returns a list of connected training environments. 
*	%logs --url: URL of the training server load balancer used to monitor the status of the training job.

The Jupyter Notebook kdapp also includes the following **cell magic** function:
*	%%training_cluster_name 
This would submit Training code to the shared Training environment

<b>%attachments</b> is a line magic command that output a table with the name(s) of the training cluster(s) available for us to use. Sometimes, tenant admin may have created multiple training clusters for different projects depending on the needs of the model or size of data, e.g. some with GPU nodes, while others with CPUs only.

In [3]:
%attachments

Training Cluster      ML Engine
--------------------  -----------
trainingengineshared  python


To utilize the training cluster, you will need grab the name of the training cluster you want to use and feed it into another custom line magic command. 

In this lab, you are going to use the **trainingengineshared** training cluster that is the shared training environment for all training jobs in the workshop. 

The Jupyter notebook will then send the contents of the cell to be executed on the training cluster. 
The example cell below will execute a print statement on the training cluster.

In [4]:
%%trainingengineshared

import datetime

print('test')
print("Date time: ", datetime.datetime.now())

History URL: http://kdss-8jsj7-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/6


The training cluster will send back a unique log url to this particular user and notebook. 
You can use this URL with another custom **line magic** command to track the status of the job in real time. 

Copy the HTTP URL output from the output of the previous cell and paste it into the cell below where it says _"your_http_url_here"_, and run the cell code.

In [14]:
##%logs --url your_http_url_here
%logs --url http://kdss-8jsj7-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/6

Job Status: Finished
test
Date time:  2020-12-16 19:10:37.100726




# Build model remotely on a shared training cluster:
Run the cell code below to build, train, test your model. At the end of this code cell, the model is saved to a file in your tenant user's working directory for later use in production deployment engine.  

Make sure you fill your <b>training cluster name</b> in the cell magic (_%%trainingengineshared_)! to execute the code cell below on the remote shared training cluster. If you comment this magic command, the code will run on your local Jupyter notebook and the training process may take longer time to complete because the local jupyter notebook is a small capacity server.

In [6]:
%%trainingengineshared

#userID = "student{{ STDID }}"

userID = "student74"

print("Importing libraries")
import pandas as pd
import numpy as np
from scipy import stats
import math
import os
import datetime
import xgboost as xgb
import pickle

import matplotlib.pyplot as plt

usecaseDirectory = "NYCTaxi"
datasetFile = "demodata.csv"
locationTable = "lookup-ipyheader.csv"
datasetFilePath = "data" + '/' + usecaseDirectory + '/' + datasetFile
locationFilePath = "data" + '/' + usecaseDirectory + '/' + locationTable
studentRepoModel = "models" + '/' + usecaseDirectory + '/' + userID

# Start time 
print("Start time for " + userID + ": ", datetime.datetime.now())

# Project repo path function
def ProjectRepo(path):
    ProjectRepo = "/bd-fs-mnt/TenantShare/repo"
    return str(ProjectRepo + '/' + path)


print("Reading in data")
# Reading in dataset table using pandas
dbName = "pqyellowtaxi"
##df = pd.read_csv(ProjectRepo('data/NYCTaxi/demodata.csv'))
df = pd.read_csv(ProjectRepo(datasetFilePath))

# Reading in latitude/longitude coordinate lookup table using pandas 
lookupDbName = "pqlookup"
##dflook = pd.read_csv(ProjectRepo('data/NYCTaxi/lookup-ipyheader.csv'))
dflook = pd.read_csv(ProjectRepo(locationFilePath))
print("Done reading in data")


# merging dataset and lookup tables on latitudes/coordinates
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.pulocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.startstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.startstationlatitude')}, inplace = True)
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.dolocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.endstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.endstationlatitude')}, inplace = True)


def fullName(colName):
    return dbName + '.' + colName

# convert string to datetime
df[fullName('tpep_pickup_datetime')] = pd.to_datetime(df[fullName('tpep_pickup_datetime')])
df[fullName('tpep_dropoff_datetime')] = pd.to_datetime(df[fullName('tpep_dropoff_datetime')])
df[fullName('duration')] = (df[fullName("tpep_dropoff_datetime")] - df[fullName("tpep_pickup_datetime")]).dt.total_seconds()

# feature engineering
# Feature engineering is the process of transforming raw data into inputs for a machine learning algorithm
df[fullName("weekday")] = (df[fullName('tpep_pickup_datetime')].dt.dayofweek < 5).astype(float)
df[fullName("hour")] = df[fullName('tpep_pickup_datetime')].dt.hour
df[fullName("work")] = (df[fullName('weekday')] == 1) & (df[fullName("hour")] >= 8) & (df[fullName("hour")] < 18)
df[fullName("month")] = df[fullName('tpep_pickup_datetime')].dt.month
# convert month to a categorical feature using one-hot encoding
df = pd.get_dummies(df, columns=[fullName("month")])

# Filter dataset to rides under 3 hours and under 150 miles to remove outliers
df = df[df[fullName('duration')] > 20]
df = df[df[fullName('duration')] < 10800]
df = df[df[fullName('trip_distance')] > 0]
df = df[df[fullName('trip_distance')] < 150]

# drop null rows
df = df.dropna(how='any',axis=0)

# select columns to be used as features
cols = [fullName('work'), fullName('startstationlatitude'), fullName('startstationlongitude'), fullName('endstationlatitude'), fullName('endstationlongitude'), fullName('trip_distance'), fullName('weekday'), fullName('hour')]
cols.extend([fullName('month_' + str(x)) for x in range(1, 7)])
cols.append(fullName('duration'))
dataset = df[cols]

# separate data into features (the taxi ride properties) and label (duration) using .iloc
X = dataset.iloc[:, 0:(len(cols) - 1)].values
y = dataset.iloc[:, (len(cols) - 1)].values
X = X.copy()
y = y.copy()
print (X.shape)
print (y.shape)
del dataset
del df

print("Done cleaning data")


print("Training and testing...")

# As we have one dataset, the data is split into a training data set and a test data set. The ideal split is 80:20. 
# 80% of the data is used to train the model and 20% is used for testing the model.
print("Load the sklearn module to split the dataset into a training data set and a test data set.")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


print("Build the model and fit the model on the training data. This will take a few minutes...")

#Define the ML algorithm (that is the learning algorithm to use). We use here the XGBRegressor class of the xgboost package to build the model
xgbr = xgb.XGBRegressor(objective ='reg:squarederror', 
                        colsample_bytree = 1,
                        subsample = 1,
                        learning_rate = 0.15,
                        booster = "gbtree",
                        max_depth = 3,
                        eta = 0.5,
                        eval_metric = "rmse",) 

print("num train elements: " + str(len(X_train)))

# then train (fit) the model on training data set comprised of inputs (features) and outputs (label)
# we provide our defined ML algorithm with data to learn from
print("Train start time: ", datetime.datetime.now())
xgbr.fit(X_train, y_train)
#after training the model, check the model training score. The closer towards 1, the better the fit.
score = xgbr.score(X_train, y_train)  
print("Model training score. The closer towards 1, the better the fit: ", score)

print("Train end time: ", datetime.datetime.now())
print("Test the model by making predictions on the test data set where only the features are provided")
y_pred = xgbr.predict(X_test)
y_pred = y_pred.clip(min=0)

print("Evaluating the model accuracy by comparing the test predictions with the test labels. RMSE is used as evaluation metric.")
# evaluating the model accuracy
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Log Error:', np.sqrt(mean_squared_log_error( y_test, y_pred)))
print()
print("Note that for RMSE the lower that value is, the better the fit")

#after we have trained the model, save it in a pickle file in the tenant user's working directory for later use in production deployment engine
#the model will be loaded back from the pickle file using the python scoring script in the deployment engine to make predictions on new data.
print("Saving model in a pickle file as " + ProjectRepo(studentRepoModel) + '/' + "XGB.pickle.dat")
pickle.dump(xgbr, open( ProjectRepo(studentRepoModel) + '/' + "XGB.pickle.dat", "wb"))

# Finish time
print("End time: ", datetime.datetime.now())

History URL: http://kdss-bwffl-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/7


### **Monitor the training job:**

#### Copy the unique log url above and paste it into the cell below to monitor your training job (repeat regularly)

The model training will take a few minutes to complete.


In [15]:
##%logs --url your_http_url_here
%logs --url  http://kdss-bwffl-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/7

Job Status: Finished
Importing libraries
Start time for student74:  2020-12-16 19:10:58.878378
Reading in data
Done reading in data
(330407, 14)
(330407,)
Done cleaning data
Training and testing...
Load the sklearn module to split the dataset into a training data set and a test data set.
Build the model and fit the model on the training data. This will take a few minutes...
num train elements: 264325
Train start time:  2020-12-16 19:11:10.809347
Model training score. The closer towards 1, the better the fit:  0.8095123420527638
Train end time:  2020-12-16 19:11:55.025369
Test the model by making predictions on the test data set where only the features are provided
Evaluating the model accuracy by comparing the test predictions with the test labels. RMSE is used as evaluation metric.
Mean Absolute Error: 195.8130179028589
Mean Squared Error: 92811.54031166565
Root Mean Squared Error: 304.64986511020425
Root Mean Squared Log Error: 0.3464091230815159

Note that for RMSE the lower that va

### **Model registry and deployment:**

##### After the model development and training, you are now ready to deploy your prediction service using the trained model, and start serving queries.

Now, let's go back to your JupyterHub account session to continue the hands-on from lab 4  
**(4-WKSHP-K8s-ML-Pipeline-Register-Model-Deployment.ipynb)** for model registry and model deployment.

You will use again your local Jupyter Notebook later...

# Retrain the model to improve model accuracy
Run the cell code below to retrain your model. The "max_depth" XGBoost parameter is set to a higher value (value=6 instead of 3) to improve the model accuracy. The model will be saved to a file as XGB.picklev2.dat

In [16]:
%%trainingengineshared

#userID = "student{{ STDID }}"

userID = "student74"

print("Importing libraries")
import pandas as pd
import numpy as np
from scipy import stats
import math
import os
import datetime
import xgboost as xgb
import pickle

import matplotlib.pyplot as plt

usecaseDirectory = "NYCTaxi"
datasetFile = "demodata.csv"
locationTable = "lookup-ipyheader.csv"
datasetFilePath = "data" + '/' + usecaseDirectory + '/' + datasetFile
locationFilePath = "data" + '/' + usecaseDirectory + '/' + locationTable
studentRepoModel = "models" + '/' + usecaseDirectory + '/' + userID

# Start time 
print("Start time for " + userID + ": ", datetime.datetime.now())

# Project repo path function
def ProjectRepo(path):
    ProjectRepo = "/bd-fs-mnt/TenantShare/repo"
    return str(ProjectRepo + '/' + path)


print("Reading in data")
# Reading in dataset table using pandas
dbName = "pqyellowtaxi"
##df = pd.read_csv(ProjectRepo('data/NYCTaxi/demodata.csv'))
df = pd.read_csv(ProjectRepo(datasetFilePath))

# Reading in latitude/longitude coordinate lookup table using pandas 
lookupDbName = "pqlookup"
##dflook = pd.read_csv(ProjectRepo('data/NYCTaxi/lookup-ipyheader.csv'))
dflook = pd.read_csv(ProjectRepo(locationFilePath))
print("Done reading in data")


# merging dataset and lookup tables on latitudes/coordinates
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.pulocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.startstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.startstationlatitude')}, inplace = True)
df = pd.merge(df, dflook[[lookupDbName + '.location_i', lookupDbName + '.long', lookupDbName + '.lat']], how='left', left_on=dbName + '.dolocationid', right_on=lookupDbName + '.location_i')
df.rename(columns = {(lookupDbName + '.long'):(dbName + '.endstationlongitude')}, inplace = True)
df.rename(columns = {(lookupDbName + '.lat'):(dbName + '.endstationlatitude')}, inplace = True)


def fullName(colName):
    return dbName + '.' + colName

# convert string to datetime
df[fullName('tpep_pickup_datetime')] = pd.to_datetime(df[fullName('tpep_pickup_datetime')])
df[fullName('tpep_dropoff_datetime')] = pd.to_datetime(df[fullName('tpep_dropoff_datetime')])
df[fullName('duration')] = (df[fullName("tpep_dropoff_datetime")] - df[fullName("tpep_pickup_datetime")]).dt.total_seconds()

# feature engineering
# Feature engineering is the process of transforming raw data into inputs for a machine learning algorithm
df[fullName("weekday")] = (df[fullName('tpep_pickup_datetime')].dt.dayofweek < 5).astype(float)
df[fullName("hour")] = df[fullName('tpep_pickup_datetime')].dt.hour
df[fullName("work")] = (df[fullName('weekday')] == 1) & (df[fullName("hour")] >= 8) & (df[fullName("hour")] < 18)
df[fullName("month")] = df[fullName('tpep_pickup_datetime')].dt.month
# convert month to a categorical feature using one-hot encoding
df = pd.get_dummies(df, columns=[fullName("month")])

# Filter dataset to rides under 3 hours and under 150 miles to remove outliers
df = df[df[fullName('duration')] > 20]
df = df[df[fullName('duration')] < 10800]
df = df[df[fullName('trip_distance')] > 0]
df = df[df[fullName('trip_distance')] < 150]

# drop null rows
df = df.dropna(how='any',axis=0)

# select columns to be used as features
cols = [fullName('work'), fullName('startstationlatitude'), fullName('startstationlongitude'), fullName('endstationlatitude'), fullName('endstationlongitude'), fullName('trip_distance'), fullName('weekday'), fullName('hour')]
cols.extend([fullName('month_' + str(x)) for x in range(1, 7)])
cols.append(fullName('duration'))
dataset = df[cols]

# separate data into features (the taxi ride properties) and label (duration) using .iloc
X = dataset.iloc[:, 0:(len(cols) - 1)].values
y = dataset.iloc[:, (len(cols) - 1)].values
X = X.copy()
y = y.copy()
print (X.shape)
print (y.shape)
del dataset
del df

print("Done cleaning data")


print("Training and testing...")

# As we have one dataset, the data is split into a training data set and a test data set. The ideal split is 80:20. 
# 80% of the data is used to train the model and 20% is used for testing the model.
print("Split dataset into a training data set and a test data set.")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


print("Build the model and fit the model on the training data. This will take a few minutes...")

#use the XGBRegressor class of the xgboost package to build the model
xgbr = xgb.XGBRegressor(objective ='reg:squarederror', 
                        colsample_bytree = 1,
                        subsample = 1,
                        learning_rate = 0.15,
                        booster = "gbtree",
                        max_depth = 6,
                        eta = 0.5,
                        eval_metric = "rmse",) 

print("num train elements: " + str(len(X_train)))

# then train (fit) the model on training data set comprised of inputs (features) and outputs (label)
print("Train start time: ", datetime.datetime.now())
xgbr.fit(X_train, y_train)
#after training the model, check the model training score. The closer towards 1, the better the fit.
score = xgbr.score(X_train, y_train)  
print("Model training score. The closer towards 1, the better the fit: ", score)

print("Train end time: ", datetime.datetime.now())
print("Test the model by making predictions on the test data set where only the features are provided")
y_pred = xgbr.predict(X_test)
y_pred = y_pred.clip(min=0)

print("Evaluating the model accuracy by comparing the test predictions with the test labels")
# evaluating the model accuracy
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Root Mean Squared Log Error:', np.sqrt(mean_squared_log_error( y_test, y_pred)))
print()
print("Note that for RMSE the lower that value is, the better the fit")


print("Saving model as " + ProjectRepo(studentRepoModel) + '/' + "XGB.picklev2.dat")
pickle.dump(xgbr, open( ProjectRepo(studentRepoModel) + '/' + "XGB.picklev2.dat", "wb"))


# Finish time
print("End time: ", datetime.datetime.now())

History URL: http://kdss-bwffl-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/9


### Monitor the training job 

#### Copy the unique log url above and paste it into the cell below to monitor your training job (repeat regularly)

The model training will take a few minutes to complete.

In [21]:
##%logs --url your_http_url_here
%logs --url http://kdss-bwffl-0.kdhs-djs2v.k8smltenant.svc.cluster.local:10001/history/9

Job Status: Finished
Importing libraries
Start time for student74:  2020-12-16 19:28:53.625921
Reading in data
Done reading in data
(330407, 14)
(330407,)
Done cleaning data
Training and testing...
Split dataset into a training data set and a test data set.
Build the model and fit the model on the training data. This will take a few minutes...
num train elements: 264325
Train start time:  2020-12-16 19:29:05.644290
Model training score. The closer towards 1, the better the fit:  0.852684422668263
Train end time:  2020-12-16 19:30:44.888571
Test the model by making predictions on the test data set where only the features are provided
Evaluating the model accuracy by comparing the test predictions with the test labels
Mean Absolute Error: 176.4289201526949
Mean Squared Error: 76454.33810087029
Root Mean Squared Error: 276.50377592515855
Root Mean Squared Log Error: 0.3103454404177983

Note that for RMSE the lower that value is, the better the fit
Saving model as /bd-fs-mnt/TenantShare/re

### Now, you can logout from your local jupyter notebook server.  

### Then go back to your JupyterHub account session to continue the hands-on from lab 5 and adjust the model registry information, section _"Dynamic? Did someone say Dynamic ML pipeline?"_.